In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image

df = pd.read_csv('/data/train.csv')
column_names = list(df.columns)

save_path = '/data/train_masks'

W = 3000
H = 3000


def create_mask(rle, w, h):
    mask = np.zeros(h*w, dtype=np.byte)
    for i in range(0, len(rle), 2):
        mask[rle[i]: rle[i] + rle[i+1]] = 1
    
    mask = np.reshape(mask, (h, w))
    return mask


for id, row in df.iterrows():
    rle = list(map(int, row['rle'].split(' ')))
    mask = create_mask(rle, W, H)

    im = Image.fromarray(mask)
    im.save(os.path.join(save_path, str(row['id']) + '.png'))
    

    

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [2]:
import os
import json

W = 3000
H = 3000
mx = 30
my = 30
bboxes = []


def create_bbox(bd, w, h, margin_x, margin_y):
    Min_Y1, Max_Y2, Min_X1, Max_X2 = h, 0, w, 0
    for pix in bd:
        Min_X1 = min(pix[0], Min_X1)
        Max_X2 = max(pix[0], Max_X2)
        Min_Y1 = min(pix[1], Min_Y1)
        Max_Y2 = max(pix[1], Max_Y2)
    
    X = max(Min_X1 - margin_x, 0)
    Y = max(Min_Y1 - margin_y, 0)
    W = min(Max_X2 + margin_x, w) - X
    H = min(Max_Y2 + margin_y, h) - Y
    
    return [ X, Y, W, H ]


ann_dir = '/data/train_annotations'
bbox_dir = '/data/train_bbox_annotations'
for ann_f in os.listdir(ann_dir):
    bboxes = []
    with open(os.path.join(ann_dir, ann_f), "r") as rf:
        ann = json.load(rf)
        for ins in ann:
            bboxes.append(create_bbox(ins, W, H, mx, my))

    with open(os.path.join(bbox_dir, ann_f), "w") as wf:
        json.dump(bboxes, wf)

